In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

First project - get some info from Japanese whisky site

get request from whysky shop page

In [ ]:
productlinks = []
for x in range(1,6):
    baseurl = "https://www.thewhiskyexchange.com"
    k = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky').text
    soup = BeautifulSoup(k, 'html.parser')
    productlist = soup.find_all('li', {'class' : 'product-grid__item'})  
    for product in productlist:
        link = product.find('a', {'class' : 'product-card'}).get('href')
        productlinks.append(baseurl + link)

get information about a product from product page

In [ ]:
data = []
for link in productlinks:
    f = requests.get(link).text
    hun = BeautifulSoup(f, 'html.parser')
    try:
        price = hun.find('p', {'class' : 'product-ation__price'}).text.replace('\n', '')
    except:
        price = None
    try:
        about = hun.find('div', {'class' : 'product-main__description'}).text.replace('\n', '')
    except:
        about = None
    try:
        rating = hun.find('div', {'class' : 'review-overview'}).text.replace('\n', '')
    except:
        rating = None
    try:
        name = hun.find('h1', {'class' : 'product-main__name'}).text.replace('\n', '')
    except:
        name = None
    whisky = {'name' : name, 'price' : price, 'rating' : rating, 'about' : about}
    data.append(whisky)

In [ ]:
data

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df

Here we try to parse Silpo page

import necessary libraries of selenium webdriver to extract javascript rendered website

In [107]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

In [108]:
# start by defining the options
options = webdriver.ChromeOptions()
options.headless = True # it's more scalable to work in headless mode
# normallly, selenium waits for all resources to download
# we don't need it as the page also populated with the running javascript code.
options.page_load_strategy = 'none'
# this returns the path webderiver downloaded
chrome_path = ChromeDriverManager().install()
chrome_service = Service(chrome_path)
# pass the driver options and service object to initialize the web driver
driver = Chrome(options=options, service=chrome_service)
driver.implicitly_wait(10)

C:\Users\Yura\AppData\Local\Temp\ipykernel_8368\2755664486.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


In [109]:
url = "https://shop.silpo.ua/category/ovocheva-konservatsiia-138"
driver.get(url)
time.sleep(10)

pageSource = driver.page_source

pageSource

element = WebDriverWait(driver, 40).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div/div[1]/div[1]/div[2]/div[2]/button"))
)

element

btn=driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[1]/div[1]/div[2]/div[2]/button');


All markdowned above is for testing

get all products

In [111]:
#content = driver.find_element(By.CSS_SELECTOR, "div[class*='product-list__wrapper']")
content = driver.find_element(By.CSS_SELECTOR, "ul[class='product-list experiment with-filter']")

In [112]:
products = content.find_elements(By.CSS_SELECTOR, "div[class='lazyload-wrapper ']")

In [113]:
products

[<selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_65")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_66")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_67")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_68")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_69")>,
 <selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60

For to do - extract image, product url, pages or number of pages, extract all elements in page

Function to extract name, price and if exists - old price

In [ ]:
def extract_data(element):
    temp_product = element.find_element(By.CSS_SELECTOR, "li[class='product-list-item-wrapper']")
    temp_product = temp_product.find_element(By.CSS_SELECTOR, "div[class='new-product-list-item']")
    product = temp_product.find_element(By.CSS_SELECTOR, "a[class='product-list-item__content']")
    try:
        name = element.find_element(By.CSS_SELECTOR, "div[class*='product-title']")
        name = product.text
    except:
        name = ''
    try:
        price = element.find_element(By.CSS_SELECTOR, "div[class*='current-price']")
        price = price.text
    except:
        price = ''
    try:
        price = element.find_element(By.CSS_SELECTOR, "div[class*='old-price']")
        old_price = price.text
    except:
        old_price = ''
    return {
        'name' : name,
        'price' : price,
        'old_price' : old_price
        
    }
    

Get list of product with prices

In [ ]:
data = []
for product in products:
    extracted_data = extract_data(product)
    data.append(extracted_data)

In [ ]:
data

For to do - extract image, product url, pages or number of pages, extract all elements in page

Current problem - only 4 products from 32 are extracted

this cell and the following ones are not important just testing

In [114]:
p = products[4]

In [115]:
p

<selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_68")>

In [116]:
product = p.find_element(By.TAG_NAME, "div")

In [117]:
product

<selenium.webdriver.remote.webelement.WebElement (session="02a6cfdae03942855ae968facc60ee04", element="FCD71007A2C0EF845F0BD997A9F3BD5A_element_94")>

In [118]:
product.tag_name

'div'

In [119]:
product.text

''

In [93]:
product.get_attribute('innerHTML')

''

https://stackoverflow.com/questions/70411736/web-scraping-gives-only-first-4-elements-on-a-page

https://www.kaggle.com/code/gauravrai2000/data-scrapping-from-a-lazy-loading-website